<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 31.0 MB/s 
     |████████████████████████████████| 182 kB 57.9 MB/s 
     |████████████████████████████████| 162 kB 64.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 162 kB 70.0 MB/s 
     |████████████████████████████████| 158 kB 72.0 MB/s 
     |████████████████████████████████| 157 kB 68.8 MB/s 
     |████████████████████████████████| 157 kB 69.2 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 53.8 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |████████████████████████████████| 157 kB 67.0 MB/s 
     |████████████████████████████████| 157 kB 72.2 MB/s 
     |████████████████████████████████| 156 kB 74.1 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [3]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [4]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [5]:
densenet121 = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_densenet121=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  densenet121,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
densenet121.trainable=False
model_densenet121.summary()

29084464/29084464 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, None, None, 1024)  7037504   
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 32)                1605664   
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 8,643,333
Trainable params: 1,605,829
Non-trainable params: 7,037,504
_____________________________________

In [6]:
model_densenet121.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [7]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "densenet121",
      notes = ""
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [8]:
%%time
epochs=15
history = model_densenet121.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/15
19/19 [==============================] - ETA: 0s - loss: 1.4069 - accuracy: 0.7365

wandb: Adding directory to artifact (/content/wandb/run-20221019_042602-260xnhx8/files/model-best)... Done. 0.4s


19/19 [==============================] - 220s 7s/step - loss: 1.4069 - accuracy: 0.7365 - val_loss: 0.1888 - val_accuracy: 0.9392
Epoch 2/15
19/19 [==============================] - ETA: 0s - loss: 0.1493 - accuracy: 0.9611

wandb: Adding directory to artifact (/content/wandb/run-20221019_042602-260xnhx8/files/model-best)... Done. 0.6s


...

19/19 [==============================] - 50s 3s/step - loss: 5.6616e-04 - accuracy: 1.0000 - val_loss: 0.0435 - val_accuracy: 0.9932
Epoch 13/15
19/19 [==============================] - ETA: 0s - loss: 5.1334e-04 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221019_042602-260xnhx8/files/model-best)... Done. 0.5s


19/19 [==============================] - 50s 3s/step - loss: 5.1334e-04 - accuracy: 1.0000 - val_loss: 0.0432 - val_accuracy: 0.9932
Epoch 14/15
19/19 [==============================] - ETA: 0s - loss: 4.6925e-04 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221019_042602-260xnhx8/files/model-best)... Done. 0.6s


19/19 [==============================] - 49s 3s/step - loss: 4.6925e-04 - accuracy: 1.0000 - val_loss: 0.0427 - val_accuracy: 0.9932
Epoch 15/15
19/19 [==============================] - 4s 171ms/step - loss: 4.3436e-04 - accuracy: 1.0000 - val_loss: 0.0429 - val_accuracy: 0.9932
CPU times: user 7min 26s, sys: 18.7 s, total: 7min 44s
Wall time: 12min 24s


In [9]:
wandb.finish()

accuracy,▁▇█████████████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▆▆▆▆▆███████
val_loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,13
best_val_loss,0.0427
epoch,14
loss,0.00043
val_accuracy,0.99324
